## Summary of data

This script allow to see the crimes from 2015-2020, and the judicaturas that had such cases

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [2]:
# Definir paths
root = Path.cwd().parent
raw = root/'data/raw'
proc = root/'data/proc'

***
### Get list of judicaturas 

There are two sources for the judicaturas codes: File sent by Consejo, and scrapped data.

First I will load the file from Consejo. It only includes the judicaturas that substanciate penal cases

In [ ]:
cods_20 = pd.read_excel(raw/'satje/0645_Id_Judicaturas.xlsx', skiprows = 5, usecols='B:C', skipfooter=5)
cods_20 = cods_20[cods_20['ID_JUDICATURA'].str.startswith('09')].copy()
cods_20['penal'] = 'penal'
cods_20.reset_index(drop=True, inplace=True)

Get list of ready judicaturas

In [ ]:
cods_15 = list((proc/'delitos_web/').glob(r'*.xls'))
cods_15 = list(map(lambda x: x.name[8:13], cods_15))
cods_15 = pd.DataFrame(cods_15, columns=['id_judicatura'])

# Limit to only Guayaquil
cods_15 = cods_15[cods_15['id_judicatura'].str.startswith('09')].reset_index(drop=True).copy()

Merge codes to see if we get all the penal judicaturas

In [ ]:
cods = pd.merge(cods_20,
    cods_15,
    how='outer',
    left_on=['ID_JUDICATURA'],
    right_on=['id_judicatura'],
    validate='1:1',
    indicator=True)

In [ ]:
cods_penal = cods.loc[cods['penal']=='penal', ['ID_JUDICATURA', 'JUDICATURA', '_merge']]
cods_penal.rename(columns={'_merge': 'descargadas'}, inplace=True)

***
### Get List of crimenes

I manually set the penalties for all the cases found in the extracted data.

In [ ]:
# Load file
causas = pd.read_excel(proc/'delitos_15_20.xlsx')

# Keep only observations that are penal
causas = causas.loc[causas['penal'].isna()]

***
### How many numbres have the secuencial

Tengo que ver cuantos # tiene el secuancial (despues de año) en cada dependencia.

Hacer un loop sobre todos los archivos. Coger las causas. Quedarnos con los numeros. Ver cuantos aparecen despues del anio.

In [ ]:
# List of files
files15 = list((proc/'delitos_web/').glob('*.xls'))

In [ ]:
def num_secuencial(df:pd.DataFrame):
    """
    Gets the number of secuencial for each year in df
    """
    # Delete casos not found
    df = df.loc[(~df['id_proceso'].isna())&(df['causa']!='No existe este proceso')].copy()
    df = df[['id_proceso']].copy()

    # Clean secuencial
    df['id_proceso'] = df['id_proceso'].str.replace('-', '')
    df['id_judicatura'] = df['id_proceso'].apply(lambda x: x[0:5])
    df['year'] = df['id_proceso'].apply(lambda x: x[5:9])
    df['secuencial'] = df['id_proceso'].apply(lambda x: x.split(x[5:9])[1])
    df['secuencial'] = df['secuencial'].str.replace('\D', '', regex=True)

    df['nsec'] = df['secuencial'].apply(lambda x: len(x))

    # Collapse by year
    df = df.groupby(by=['id_judicatura', 'year'], as_index=False).agg(min=('nsec', 'min'), max=('nsec', 'max'))

    return df

In [ ]:
num_sec = pd.DataFrame()
for file in files15:
    df = pd.read_excel(file, dtype={'id_proceso': str})
    num_sec = pd.concat([num_sec, num_secuencial(df)], ignore_index=True)

In [ ]:
# Contar cuantos anos tengo para cada judicatura
nyear = num_sec.groupby(['id_judicatura'], as_index=False).agg(nyear=('year', 'count'))
num_sec = num_sec.merge(nyear)

***
### Obtener listas de cosas a webscrapear

1. Dependencias que no se logro 2015
2. Dependencias que no teniamos en la lista
3. Sentencias que se cortaron
4. Las de 2014 hacia atras

**1. No se saco 2015**

In [ ]:
# Keep only for guayas
guayas = pd.merge(cods_penal, num_sec, how='left', left_on='ID_JUDICATURA', right_on='id_judicatura', validate='1:m', indicator=True)

lista2015 = list(guayas.loc[(guayas['nyear']<=5)&(guayas['year']=='2016'), 'id_judicatura'])

In [ ]:
pd.DataFrame(lista2015, columns=['id_judicatura']).to_csv(proc/'restantes_2015.csv', index=False)

**2. Dependencias no en la lista, pero penal de guayas**

In [ ]:
restantesgye = list(cods_penal.loc[cods_penal['descargadas']!='both', 'ID_JUDICATURA'])
pd.DataFrame(restantesgye, columns=['id_judicatura']).to_csv(proc/'restantes_gye.csv', index=False)

**3. Sentencias que se cortaton**

In [ ]:
files = list((proc/'delitos_web/old_delitos').glob('*.xls'))

In [ ]:
celdas_incompletos = pd.DataFrame()
for file in files:
    df = pd.read_excel(file)
    if 'sentencia' in df.columns:
        df['size_sentencia'] = df.loc[~df['sentencia'].isna(), 'sentencia'].apply(lambda x: len(x))
        df = df.loc[~df['size_sentencia'].isna(), ['size_sentencia', 'id_proceso']].copy()

        celdas_incompletos = pd.concat([celdas_incompletos, df], ignore_index=True)



In [ ]:
# Find those that are limited
(celdas_incompletos
 .loc[(celdas_incompletos['size_sentencia']>=32767) & (celdas_incompletos['id_proceso'].str.startswith('09')),
        ['id_proceso']]
 .to_csv(proc/'procesos_incompletos.csv', index=False))


**4. Procesos 2014 para atras**

In [3]:
# Get list of infracciones para ver cuales son penales
files = list((proc/'resumenes/by_judicatura').glob('*.csv'))

In [4]:
def find_nth_overlapping(haystack, needle, n):
    matches = list(re.finditer(needle, haystack))

    if len(matches) < n:
        return -1
    return matches[n-1].start()

In [5]:
def no_digit_start(txt:str):
    """
    Return text after first no giit element in ```txt```
    """

    match = re.search('\D', txt)
    if match:
        return txt[match.start():]

    else:
        return ""

In [6]:
def acciones(txt:str):

    txt = txt.lstrip('-')

    match = re.search('\D', txt)

    if match:
        fnodigit = txt[match.start()]

        # Si el primero es espacio en blanco
        if fnodigit == " ":
            return {'sec': txt[:match.start()] , 'accion': txt[match.start()+1:]}

        else:
            txt2 = txt[match.start()+1:]
            match2 = re.search('\D', txt2)

            if match2:
                snodigit = txt2[match2.start()]

                if snodigit == ' ':
                    return {'sec':txt[:match.start()+2].rstrip(' '), 'accion': txt2[match2.start()+1:]}

                else:
                    return {'sec':txt[:match.start()+1].rstrip(' '), 'accion': txt2[match2.start():]}

            else:
                return {'sec': txt, 'accion': 'No hay caso'}
    else:
        return {'sec': txt, 'accion': 'No hay caso'}

In [87]:
files_10_14 = pd.DataFrame()

for file in files:

    # Check if file has data
    df = pd.read_csv(file, dtype={'causa': str})
    df = df.loc[df['descripcion']!='No se encuentran resultados.'].copy()
    df = df.loc[~df['descripcion'].isna()].copy()

    # Extract year
    df['year'] = df['descripcion'].apply(lambda t: t[t.find('-')+1:find_nth_overlapping(t, '-', 2)])

    # Extract fecha
    df['fecha'] = df['descripcion'].apply(lambda t: t[t.find('/')-2:find_nth_overlapping(t, '/', 2)+4+1])

    # Id_judicatura
    df['id_judicatura'] = df['descripcion'].apply(lambda t: t[find_nth_overlapping(t, '/', 2)+5:t.find('-')])

    # Restante
    df['restante'] = df['descripcion'].apply(lambda t: t[find_nth_overlapping(t, '-', 2):])
    df['accion'] = df['restante'].apply(lambda t: acciones(t)['accion'].lstrip(' '))
    df['id_sec'] = df['restante'].apply(lambda t: acciones(t)['sec'])

    # Extract id_proceso + accion
    df['proceso_long'] = df['descripcion'].apply(lambda t: t[find_nth_overlapping(t, '/', 2)+5:])
    df['proceso_long'] = df['proceso_long'].str.replace('-', '')
    df['proceso_long'] = df['proceso_long'].apply(lambda t: t[:15])
    df['proceso_long'] = df['proceso_long'].str.rstrip(' ')

    # Store results
    files_10_14 = pd.concat([files_10_14, df], ignore_index=True)


In [88]:
files_10_14.drop_duplicates(ignore_index=True, inplace=True)

See if the acciones are criminal or not. This part I did manually

In [89]:
# Load file with crimes
crimenes_1014 = pd.read_excel(proc/'delitos_lists/delitos_2010_2014_gye.xlsx')
crimenes_1014 = crimenes_1014[crimenes_1014['eliminar'].isna()].copy()
crimenes_1014.drop(columns='eliminar', inplace=True)

In [90]:
# Select acciones
to_scrap = pd.merge(files_10_14,
            crimenes_1014,
            how = 'outer',
            on = 'accion',
            validate='m:1',
            indicator=True)

In [91]:
to_scrap = (to_scrap
            .loc[to_scrap['_merge']=='both']
            .drop(columns='_merge')
            .reset_index(drop=True))

In [98]:
# Save data
to_scrap = (to_scrap[['id_judicatura', 'year', 'id_sec', 'accion', 'proceso_long']]
            .sort_values(by=['id_judicatura', 'year', 'id_sec'], ignore_index=True))

for judicatura in list(['09111', '09112']):

    df = to_scrap.loc[to_scrap['id_judicatura'] == judicatura].copy()
    df['estado'] = 0
    df['id_matched'] = np.nan
    df.to_excel(proc/f'scrap_lists/2010_2014/{judicatura}.xlsx', index=False)

In [95]:
# create lista to keep track
to_scrap[['id_judicatura']].drop_duplicates(ignore_index=True).to_excel("C:/Users/DanielJaramillo/Dropbox/Research/estado_2010_2014.xlsx", index=False)

***

In [52]:
def completar(iddep:str):
    df = pd.read_csv(proc/f'delitos_web/resumen_{iddep}.csv',
            dtype={'id_proceso': str})

    idproc = df.loc[df.shape[0]-1, 'id_proceso']
    idproc = re.sub('\D', '', idproc)

    if idproc[-1] == '9':
        decenas = int(idproc[-2]) + 1
        new_proc = idproc[:-2] + str(decenas) + '0'

    else:
        ldigit = int(idproc[-1]) + 1
        new_proc = idproc[:-1] + str(ldigit)

    return pd.concat([df, pd.DataFrame({'id_proceso': [new_proc]})], ignore_index=True)

In [101]:
iddep = '09286'

In [102]:
completar(iddep)

,causa,demandado,demandante,id_proceso
0,"394 CONTRAVENCIONES DE SEGUNDA CLASE, INC.1, N...",JUVENAL CRUZ MARTILLO DIAZ,PPNN,0928620150001
1,385 CONDUCCIÃÂN DE VEHÃÂCULO EN ESTADO DE ...,VIZCAINO QUINTO ADRIANA IVANOVA,PARTE DE TRANSITO CTE,0928620150001G
2,"393 CONTRAVENCIONES DE PRIMERA CLASE, INC.1, N...",RODRIGUEZ LARAFERNANDO ENRIQUE RODRIGUEZ LARA,PPNN,0928620150002
3,379 LESIONES CAUSADAS POR ACCIDENTE DE TRÃÂN...,GARCIA SALVATIERRA KLEBERGARCIA SALVATIERRA KL...,TORRES VELIZ IVONNE ALEXANDRAAB. REYNALDO CEVA...,0928620150003
4,"393 CONTRAVENCIONES DE PRIMERA CLASE, INC.1, N...",ZARAY NOEMI ASCENCIO ANASTACIO,TOLA ANTEPARA BYRON CRISTIAN,0928620150004
...,...,...,...,...
17874,ART. 230 # 4 COFJ EXCARCELACIÓN POR CUMPLIMIEN...,NaN,NaN,09286201702503
17875,"396 CONTRAVENCIONES DE CUARTA CLASE, INC.1, NU...",CARMEN MAGALY CAAMAÑO CORNEJO,RUMIÑAHUI HERRERA YENNY MERCEDES,09286201702504
17876,ART. 230 # 10 COFJ-ART. 632 COIP- CONTROL DE C...,NaN,NaN,09286201702505
17877,ART. 230 # 10 COFJ-ART. 632 COIP- CONTROL DE C...,NaN,NaN,09286201702506


In [103]:
completar(iddep).to_csv(proc/f'delitos_web/resumen_{iddep}.csv', index=False)

In [ ]:
df_start = pd.read_csv(proc/f'delitos_web/resumen_{iddep}.csv', dtype={'id_proceso': str})

In [ ]:
df.to_csv(proc/f'delitos_web/resumen_{iddep}.csv', index=False)

In [142]:
def complete_procesos(iddep):
    df = pd.read_excel(proc/f'scrap_lists/2010_2014/{iddep}.xlsx',
        dtype={'id_judicatura': str, 'id_sec': str, 'year': str, 'id_matched':str, 'proceso_long':str})
    idx_change = df[df['estado']==0].index[0]
    df.loc[idx_change, 'estado'] = 1

    return df

In [143]:
iddep = '09122'

In [146]:
complete_procesos(iddep)

In [148]:
complete_procesos(iddep).to_excel(proc/f'scrap_lists/2010_2014/{iddep}.xlsx', index=False)

***


**To Copy files**

In [140]:
for folder in Path('C:/Users/DanielJaramillo/Desktop/files').iterdir():

    id_proceso = folder.stem

    try:
        folder.rename(proc/f'delitos_web/files/{id_proceso}')
    except FileExistsError:
        pass
        

**Delete empty folders**

In [ ]:
for folder in (proc/'delitos_web/files').iterdir():
    try: 
        folder.rmdir()
    except:
        pass